# 介绍

这个demo直接加载模型到进程进行调用，不部署服务。

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
model_dir = "/share/luanhongzhi/Qwen1.5-14B-Chat-GPTQ-Int4"
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

CUDA extension not installed.
CUDA extension not installed.
/usr/local/miniconda3/envs/autogen/lib/python3.10/site-packages/transformers/modeling_utils.py:4371: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
import torch
def qwen_chat(prompt):
    device = "cuda:0" # the device to load the model onto
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    input_ids = tokenizer.encode(text,return_tensors='pt')
    attention_mask = torch.ones(input_ids.shape,dtype=torch.long,device=device)
    
    generated_ids = model.generate(
        model_inputs.input_ids,
        attention_mask=attention_mask,
        max_new_tokens=512,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [4]:
qwen_chat("你好")

'你好！很高兴为你提供帮助。有什么可以帮你的吗？'

In [ ]:
qwen_chat("作为一个程序员，应该如何高效的管理时间和安排工作，提高工作效率？")